<a href="https://polly.elucidata.io/manage/workspaces?action=open_polly_notebook&amp;source=github&amp;path=ElucidataInc%2Fpolly-python%2Fblob%2Fmain%2FIngest%2Ffind_user_roles_in_all_omixatlas.ipynb&amp;kernel=elucidata%2FPython+3.10&amp;machine=medium" target="_parent"><img alt="Open in Polly" src="https://elucidatainc.github.io/PublicAssets/open_polly.svg"/></a>


In [ ]:
import io
import os
import json
import random
from pathlib import Path
from time import time

import boto3
import requests
import pandas as pd
from requests import Session

STEPS TO GET THE AUTH KEY:
1. Go to Polly Dashboard
2. Go to User Options > Authentication (In the bottom right of the screen).
3. Go to Authentication Keys tab
4. Enter the name and description of the key and GENERATE the KEY!
5. Copy the key generated and paste it below.

In [ ]:
token = "<YOUR AUTH KEY HERE>"

In [ ]:
class UnauthorizedException(Exception):
    def __str__(self):
        return f"Authorization failed"


class PollySession(Session):
    def __init__(self, token):
        Session.__init__(self)
        self.headers = {
            "Content-Type": "application/vnd.api+json",
            "x-api-key": token,
            "User-Agent": "jupyter-notebook"
        }


class Polly:
    default_session = None

    @classmethod
    def auth(cls, token):
        cls.default_session = PollySession(token)

    @classmethod
    def get_session(cls, token=None):
        if not token:
            if not cls.default_session:
                raise UnauthorizedException
            else:
                return cls.default_session
        else:
            return PollySession(token)


In [ ]:
session = Polly.get_session(token)

In [ ]:
resp = session.get("https://v2.api.polly.elucidata.io/users/me")

if resp.status_code == 401:
    print("This user is unauthorized, or the AUTHKEY is invalid")

elif resp.status_code == 200:
    user_response = resp.json()
    
else:
    try:
        print(resp.text)
    except:
        print("something went wrong. status code:", res.status_code)

In [ ]:
uid = user_response.get("data").get("id")

In [ ]:
repositories_response = session.get("https://api.discover.polly.elucidata.io/repositories")
repositories = []
if(repositories_response.status_code == 200):
    repositories = repositories_response.json()['data'] 
else:
    print(repositories_response.text)

In [ ]:
repository_ids = []
repo_name_map = {}

for repo in repositories:
    rid = repo['id']
    repository_ids.append(rid)
    repo_name_map[rid] = repo['attributes']['frontend_info']['display_name']

In [ ]:
def get_user_roles(repo_id):
    #find user role for repo_id
    user_role_response = session.get(f"https://api.discover.polly.elucidata.io/repositories/{repo_id}/users/{uid}")
    if user_role_response.status_code == 200:
        user_data = user_role_response.json()['data']
        return {'repo_id':repo_id,'user_data':user_data}
    else:
        return None
    

In [ ]:
import concurrent.futures

datas = []
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_url = {executor.submit(get_user_roles,repo_id):repo_id for repo_id in repository_ids}
    for future in concurrent.futures.as_completed(future_to_url):
        repo_id = future_to_url[future]
        try:
            data = future.result()
            if data is not None:
                datas.append(data)
        except Exception as e:
            print(str(e))



In [ ]:
if len(datas) == 0:
    print("This user does not have access to any repositories!")
else:
    i = 1
    print(" ","  {:<15}  {:<30}  {:<20}".format("RepoID","RepoName","Role"))
    print("-"*69)
    for data in datas:
        repo_id = data['repo_id']
        user_data = data['user_data']
        print(i,"  {:<15}  {:<30}  {:<20}".format(repo_id,repo_name_map[repo_id],user_data['attributes']['role']))
        i = i + 1